In [1]:
from sklearn.metrics import classification_report, r2_score, mean_absolute_error, mean_squared_error, confusion_matrix
from keras.layers import Dense, Dropout, Activation
from multiprocessing import Pool, cpu_count
from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn.ensemble import ExtraTreesClassifier
from xgboost import XGBRegressor
import random
from datetime import timedelta
import statistics
from datetime import datetime
from keras.models import model_from_json
import warnings
from keras.optimizers import RMSprop, Adam
import pandas as pd
from keras.activations import relu
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler
import keras
from tensorflow import set_random_seed
import numpy as np
from sklearn.model_selection import train_test_split
from numpy.random import seed
import os
import tensorflow as tf
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
SEED = 69
seed(69)
set_random_seed(69)
np.random.seed(69)
warnings.filterwarnings("ignore")


num_pool_to_use = cpu_count() - 1
week_df = pd.read_csv("Week_df_features.csv", index_col=0)
week_df['date'] = pd.to_datetime(week_df['date'])

Using TensorFlow backend.


In [3]:
# Considering 2015,16 and 17 data only


train_clas = week_df[(week_df['date'].dt.year) != 2017]
test_clas = week_df[(week_df['date'].dt.year) == 2017]
train = week_df[(week_df['date'].dt.year) != 2017]
test = week_df[(week_df['date'].dt.year) == 2017]
df_2015 = week_df[(week_df['date'].dt.year) == 2015]
df_2016 = week_df[(week_df['date'].dt.year) == 2016]
df_2017 = week_df[(week_df['date'].dt.year) == 2017]

train_2015, test_2015 = train_test_split(
    df_2015, test_size=0.3, shuffle=True, random_state=69)
train_2016, test_2016 = train_test_split(
    df_2016, test_size=0.3, shuffle=True, random_state=69)
train_2017, test_2017 = train_test_split(
    df_2017, test_size=0.3, shuffle=True, random_state=69)
train_clas = pd.concat([train_2015, train_2016, train_2017])
test_clas = pd.concat([test_2015, test_2016, test_2017])
train = pd.concat([train_2015, train_2016, train_2017])
test = pd.concat([test_2015, test_2016, test_2017])

In [4]:
train.head()

,WeekOfShow,Movie,ReleaseWeek,LastWeek,Lifetime,WeeksSinceRelease,LastDate,ReleaseDate,OccAtWeek,OccAtPred,...,running_days_in_next_week,Total_MoviesinWeek,Target_Clas_1/7,Target_Clas_2/7,Target_Clas_3/7,Target_Clas_4/7,Target_Clas_5/7,Target_Clas_6/7,Target_Clas_7/7,WeekOfTheYear
960,84,HO00001104,84,85,1,0,2015-10-29,2015-10-21,1267,563,...,2,26,1.0,1.0,0.0,0.0,0.0,0.0,0.0,44
583,58,HO00000878,56,58,2,2,2015-04-23,2015-04-10,143,22,...,0,21,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18
1005,85,HO00001134,85,86,1,0,2015-11-09,2015-10-31,380,145,...,6,37,1.0,1.0,1.0,1.0,1.0,1.0,0.0,45
823,68,HO00000943,65,69,4,3,2015-07-09,2015-06-12,1968,633,...,2,31,1.0,1.0,0.0,0.0,0.0,0.0,0.0,28
507,55,HO00000814,50,55,5,5,2015-04-01,2015-02-27,220,32,...,0,27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15


In [5]:
features = [
    'WeeksSinceRelease',
    'Capacity', 'Cap_110', 'Cap_120', 'Cap_131', 'Cap_310',
    'ShowsInWeek', 'BLUSH', 'CARVE', 'FRAME', 'KITES', 'PLUSH', 'SPOT', 'STREAK',
    'WEAVE', 'trans_1', 'trans_2', 'trans_3', 'trans_4', 'trans_5',
    'trans_6', 'trans_7', 'trans_8', 'trans_9', 'trans_10', 'trans_11',
    'trans_12', 'OccCumSum', 'CapCumSum', 'OccCumSumPer',
    'OtherReleasesInWeek', 'Walkin', 'WalkinPer', 'OccPer',
    'Month',
    'ShowHistory0', 'ShowHistory1', 'ShowHistory2',
    'ShowHistory3', 'ShowHistory4', 'ShowHistory5', 'ShowHistory6',
    'WalkinHistory0', 'WalkinHistory1', 'WalkinHistory_Mean',
    'WalkinHistory_Max', 'WalkinHistory_Min', 'History_Mean',
    'WalkinHistory2', 'WalkinHistory3', 'WalkinHistory4', 'WalkinHistory5',
    'WalkinHistory6', 'SeatsSoldPerTrans0',
    'SeatsSoldPerTrans1', 'SeatsSoldPerTrans2', 'SeatsSoldPerTrans3',
    'SeatsSoldPerTrans4', 'SeatsSoldPerTrans5', 'SeatsSoldPerTrans6',
    'SeatsSoldPerTrans_History_Mean', 'SeatsSoldPerTrans_History_Max',
    'SeatsSoldPerTrans_History_Min', 'RelativeOccAtWeek0',
    'RelativeOccAtWeek1', 'RelativeOccAtWeek2', 'RelativeOccAtWeek3',
    'RelativeOccAtWeek4', 'RelativeOccAtWeek5', 'RelativeOccAtWeek6',
    'RelativeOccAtWeek_History_Mean', 'RelativeOccAtWeek_History_Max',
    'RelativeOccAtWeek_History_Min', 'Total_MoviesinWeek', 'Drama', 'Comedy', 'Romance', 'Runtime', 'Language']

# Best Two Stage-ET+DNN

In [6]:
def load_model():
    json_file = open('/mnt/home/spicrowdpred/Spi-Occ/Lifetime/model.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights(
        "/mnt/home/spicrowdpred/Spi-Occ/Lifetime/model.h5")
    print(f"Loaded model from disk")
    return loaded_model

In [9]:
from sklearn.ensemble import ExtraTreesClassifier
usefulFeatures = ['Capacity', 'DaysShowedInWeek', 'Total_MoviesinWeek',
                  'WeekOfTheYear', 'ShowsInWeek', 'ShowHistory1', 'RelativeOccAtWeek0',
                  'trans_2', 'trans_3', 'OccPer', 'RelativeOccAtWeek_History_Min']
print(len(usefulFeatures), "is length of importantfeatures")
etc = ExtraTreesClassifier(n_estimators=200, min_samples_split=3,
                           min_samples_leaf=1, max_depth=None, bootstrap=False, n_jobs=-1)
etc.fit(train_clas[usefulFeatures], train_clas['Target_Clas'])

predictions_clas = (etc.predict(test_clas[usefulFeatures]))

print(classification_report(test_clas["Target_Clas"], predictions_clas))
print(confusion_matrix(test_clas["Target_Clas"], predictions_clas))
classification_confusion_matrix = classification_report(
    test_clas["Target_Clas"], predictions_clas)
test_clas['predicted'] = predictions_clas
test['Clas_Pred'] = predictions_clas
test_reg = test_clas[test_clas['predicted'] == 1]
train_reg = train[train['Target'] > 0]

seed(69)
set_random_seed(69)
np.random.seed(69)

test_clas['predicted'] = predictions_clas
test['Clas_Pred'] = predictions_clas
test_reg = test_clas[test_clas['predicted'] == 1]

test_reg = test_reg[test_reg['week_x'] != 192]

train_reg = train[train['Target'] > 0]

model = Sequential()
model.add(Dense(128, input_shape=(train_reg[features].shape[-1],)))
model.add(keras.layers.LeakyReLU())
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(64))
model.add(keras.layers.LeakyReLU())
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(32))
model.add(keras.layers.LeakyReLU())
model.add(BatchNormalization())
model.add(Dense(8))
model.add(keras.layers.LeakyReLU())
model.add(BatchNormalization())
model.add(Dense(1,))
opzr = RMSprop(lr=3e-3)
model.compile(optimizer=opzr, loss='mae')  # mae has 60% no error


scalerX = StandardScaler().fit(train_reg[features])  # Standard Scalar
scalery = StandardScaler().fit(np.array(train_reg['Target']).reshape(-1, 1))
X_train = scalerX.transform(train_reg[features])
y_train = scalery.transform(np.array(train_reg['Target']).reshape(-1, 1))
X_test = scalerX.transform(test_reg[features])
y_test = scalery.transform(np.array(test_reg['Target']).reshape(-1, 1))
seed(69)
set_random_seed(69)
np.random.seed(69)

# model.fit(X_train,y_train,epochs=14,batch_size=32,verbose=1)
model = load_model()
ann_pred = (model.predict(scalerX.transform(test_reg[features].values)))
ann_pred = scalery.inverse_transform(ann_pred)
print("R SQURED SCORE      :", r2_score(test_reg["Target"], ann_pred))
print("MAE                 :", mean_absolute_error(
    test_reg["Target"].values, ann_pred))
print("MSE                 :", mean_squared_error(
    test_reg["Target"].values, ann_pred))
print("RMSE                :", np.sqrt(
    mean_squared_error(test_reg["Target"].values, ann_pred)))
ann_error_df = pd.DataFrame()
ann_error_df = test_reg[["Movie", "Target",
                         "Capacity", 'Lifetime', 'Language']]
ann_error_df['Pred'] = ann_pred
ann_error_df["error"] = (abs(ann_error_df["Target"]-ann_error_df["Pred"]))
ann_error_df.loc[ann_error_df["error"] < 1, "count"] = '0 Week Error:'
ann_error_df.loc[((ann_error_df["error"] >= 1) & (
    ann_error_df["error"] < 2)), "count"] = '1 Week Error:'
ann_error_df.loc[((ann_error_df["error"] >= 2) & (
    ann_error_df["error"] < 3)), "count"] = '2 Week Error:'
ann_error_df.loc[((ann_error_df["error"] >= 3) & (
    ann_error_df["error"] < 4)), "count"] = '3 Week Error:'
ann_error_df.loc[ann_error_df["error"] >= 4, "count"] = '>3 Week Error:'
ann_error = ann_error_df["count"].value_counts()
ann_error = (ann_error*100)/sum(ann_error)
print(ann_error)

11 is length of importantfeatures
              precision    recall  f1-score   support

         0.0       0.97      0.92      0.94       312
         1.0       0.97      0.99      0.98       743

   micro avg       0.97      0.97      0.97      1055
   macro avg       0.97      0.95      0.96      1055
weighted avg       0.97      0.97      0.97      1055

[[288  24]
 [ 10 733]]
Loaded model from disk
R SQURED SCORE      : 0.3073924663550127
MAE                 : 1.1610977039413048
MSE                 : 4.67174451076025
RMSE                : 2.16142187246272
0 Week Error:     64.854111
1 Week Error:     19.098143
2 Week Error:      7.824934
>3 Week Error:     4.774536
3 Week Error:      3.448276
Name: count, dtype: float64


## THRESHOLD (DNN)

In [5]:
from sklearn.metrics import confusion_matrix, classification_report
test['predicted'] = predictions

test.loc[test["predicted"] >= (1/7), "predicted_Clas_1"] = 1
test.loc[test["predicted"] < (1/7), "predicted_Clas_1"] = 0

test.loc[test["predicted"] >= (2/7), "predicted_Clas_2"] = 1
test.loc[test["predicted"] < (2/7), "predicted_Clas_2"] = 0

test.loc[test["predicted"] >= (3/7), "predicted_Clas_3"] = 1
test.loc[test["predicted"] < (3/7), "predicted_Clas_3"] = 0


test.loc[test["predicted"] >= (4/7), "predicted_Clas_4"] = 1
test.loc[test["predicted"] < (4/7), "predicted_Clas_4"] = 0

test.loc[test["predicted"] >= (5/7), "predicted_Clas_5"] = 1
test.loc[test["predicted"] < (5/7), "predicted_Clas_5"] = 0

test.loc[test["predicted"] >= (6/7), "predicted_Clas_6"] = 1
test.loc[test["predicted"] < (6/7), "predicted_Clas_6"] = 0

test.loc[test["predicted"] >= (7/7), "predicted_Clas_7"] = 1
test.loc[test["predicted"] < (7/7), "predicted_Clas_7"] = 0


test.loc[test["Target"] > 0, "Target_Class"] = 1
test.loc[test["Target"] <= 0, "Target_Class"] = 0

confusion_matrices = []
for i in range(1, 8):
    confusion_matrices.append(confusion_matrix(
        test["Target_Class"], test["predicted_Clas_"+str(i)]))
    # print(confusion_matrices[i-1])
    # print(confusion_matrices[i-1][0][0])
    print("recall for 0 as", i, "/7")
    print(confusion_matrices[i-1][0][0]/sum(confusion_matrices[i-1][0]))
    print(sum(confusion_matrices[i-1][0]))
#     print("*****-------------------*************")
#     print("classification results---------------------------------")
#     print(classification_confusion_matrix)
#     print("-----------------end------------------------------------")
    print(confusion_matrices[i-1])
    print(classification_report(
        test["Target_Class"], test["predicted_Clas_"+str(i)]))
    # print(confusion_matrices(test["Target_Class"],test["predicted_Clas_"+str(i)]))
    print
    print("------------------------------")

recall for 0 as 1 /7
0.7788461538461539
312
[[243  69]
 [  8 735]]
              precision    recall  f1-score   support

         0.0       0.97      0.78      0.86       312
         1.0       0.91      0.99      0.95       743

   micro avg       0.93      0.93      0.93      1055
   macro avg       0.94      0.88      0.91      1055
weighted avg       0.93      0.93      0.92      1055

------------------------------
recall for 0 as 2 /7
0.8525641025641025
312
[[266  46]
 [ 12 731]]
              precision    recall  f1-score   support

         0.0       0.96      0.85      0.90       312
         1.0       0.94      0.98      0.96       743

   micro avg       0.95      0.95      0.95      1055
   macro avg       0.95      0.92      0.93      1055
weighted avg       0.95      0.95      0.94      1055

------------------------------
recall for 0 as 3 /7
0.875
312
[[273  39]
 [ 15 728]]
              precision    recall  f1-score   support

         0.0       0.95      0.88      0.

In [ ]:
# Similarly Thresholds were obtained for XGB and ET